In [ ]:
print("Loading data module...")

In [ ]:
def create_dataframe(filepath, columns, nrows=1e100):
    # Load data from an OpenSecrets csv file and
    # clean up the data consistently.

    print("Creating dataframe...")

    columns = dict(enumerate(columns))
    df = pd.read_csv(filepath, nrows=nrows, on_bad_lines='skip', sep=',', header=None, 
                     na_values=['N/A', 'NA'], encoding='ISO-8859-1')
    df = df.rename(columns=columns)
    
    # Remove pipes surrounding data.
    df = df.applymap(lambda x: x.replace('|', '') if type(x) == str else x)

    # Type casting. Do not include EINs, some are alphnumeric for candidates.
    df['active'] = df['active'].apply(lambda x: int(x)) if 'active' in df.columns else None #cmtes22
    df['cycle'] = df['cycle'].apply(lambda x: int(x)) if 'cycle' in df.columns else None #cands22, cmtes527
    df['fecrecno'] = df['fecrecno'].apply(lambda x: int(x)) if 'fecrecno' in df.columns else None #df_pac_other22
    df['fectransid'] = df['fectransid'].apply(lambda x: int(x)) if 'fectransid' in df.columns else None # indivs22
    df['foreign'] = df['foreign'].apply(lambda x: int(x)) if 'foreign' in df.columns else None #cmtes22

    # Other cleanup.
    filename = os.path.basename(filepath)
    
    if 'date' in df.columns:
        if filename == 'indivs22.csv':
            df['date'] = df['date'][:9]
        df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')
        if filename == 'pac_other22.csv':
            df = df.dropna(subset=['date'])

    if 'microfilm' in df.columns:
        df['microfilm'] = df['microfilm'].fillna(0)
        df['microfilm'] = df['microfilm'].apply(lambda x: 0 if x == '' or not str(x).isdigit() else int(x))

    print("dataframe created...")
    
    return df

In [ ]:
print("...data module loaded.")